En este notebook se hara el proceso de **PRIORIZAR** , se tomaran las PQR Desde diciembre del 2022 a  **2023 a Marzo(periodo de tres meses)**


# CONFIGURAR SPARK

In [68]:
import pandas as pd
import sys
sys.path.append('../librerias')


# LIBRERIAS





In [2]:
!python -m spacy download es_core_news_sm

2023-06-23 02:01:31.308967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-23 02:01:32.442560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 84.6 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [3]:
import pandas as pd

from google.colab import drive
import gc

import seaborn as sns
import plotly.express as px

import numpy as np

import sqlite3
from sqlite3 import Error
import os
import re, string
from nltk.corpus import stopwords
import nltk
import matplotlib.pyplot as plt

import torch
#from transformers import AutoTokenizer, BertForSequenceClassification




nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


import pandas as pd
import re, string


pd.options.display.max_columns = None

#from sentence_transformers import SentenceTransformer, util
#from transformers import *

import spacy

nlp = spacy.load('es_core_news_sm', disable=['tagger', 'parser', 'ner'])



from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import preprocessing


#.master("local[2]")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
%pip install pycorrector
%pip install unidecode
%pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 39.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 37.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycorrector: filename=pycorrector-0.5.0-py3-none-any.whl size=4480931 sha256=2ef5512a90ad5f10eb7cfcf52ef6ebae9198b3ca5f910eed95fd693b7d2fcdee
  Stored in directory: /root/.cache/pip/wheels/03/cd/34/3c82eb07a21023abd3b8c334a96cb2b8f32cdd986d0f7d0bf4
  Created wheel for kenlm: filename=kenlm-0.1-cp310-cp310-linux_x86_64.whl size=3003969 sha256=8d5dc9d0bdcd690ddf5ca4b828449657c2152a3b930de8d0a30969a3abad8cc7
  Stored in directory: /root/.cache/pip/wheels/4e/3a/01/9105a

# ETL

In [7]:

pandas_c_F= pd.read_csv('/content/PRIORIZAR/entranamiento_/binario_entrenamiento_cruda.csv').sort_values('NUMERO_PQR', ascending=False)

pandas_c_F= pandas_c_F.fillna("vacios")


pandas_c_F

,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,clasificacion
8511,8511,2023-02-28 23:37:26,MDM-PQR-37404310,FREDY ASCANIO ASCANIO Cédula de Ciudadanía 881...,1
16731,16731,2023-02-28 22:30:53,MDM-PQR-37404041,SE COMUNICA POR CONCEPTOS FACTURABLES Y VALOR ...,0
16730,16730,2023-02-28 22:02:54,MDM-PQR-37403846,FECHA Y HORA DE GESTION :28/02/2023/22:00 LI...,0
16729,16729,2023-02-28 21:55:07,MDM-PQR-37403781,ct solicita envio de factura se envia documento,0
16728,16728,2023-02-28 21:46:59,MDM-PQR-37403700,nombre: ANA GLADYS PARADA VARGAS cc: 35464758 ...,0
...,...,...,...,...,...
0,0,2022-12-01 05:57:44,MDM-PQR-35424576,Linea: 6013842997 Nombre: LINA MARIA CESPEDES ...,1
8515,8515,2022-12-01 05:24:58,MDM-PQR-35424468,SG//LT: 6013835268 cuenta 12053820540// client...,0
8514,8514,2022-12-01 04:48:27,MDM-PQR-35424385,SG//LT: 6017257398 cuenta 12054364769// client...,0
8513,8513,2022-12-01 04:36:02,MDM-PQR-35424371,SG//LT: 6016133556 cuenta 12054410753// client...,0


In [8]:
pandas_c_F.isnull().sum()

index              0
FECHA_CREACION     0
NUMERO_PQR         0
DESCRIPCION_PQR    0
clasificacion      0
dtype: int64

In [5]:
from clean_text import clean_text

In [84]:
import pandas as pd
import re

### prueba

print('entro2')
 
def clean_text(text):
    
    text = text.lower()
    
    text=re.sub(r'cliene', "cliente", text)

    text=re.sub(r".*// cliente", " ", text, flags=re.IGNORECASE)

    text=re.sub(r".*com", " ", text, flags=re.IGNORECASE)
    text=re.sub(r".*com", " ", text, flags=re.IGNORECASE)
    

    text=re.sub(r".*//cliente", " ", text, flags=re.IGNORECASE)


    text=re.sub(r'No exitoso - RF-', "otros", text) 
    
    text = re.sub(r'[^\w\s]', '', re.sub(r'[áéíóú]', lambda m: {'á':'a', 'é':'e', 'í':'i', 'ó':'o', 'ú':'u'}[m.group(0)], text))
  
    text =re.sub(r'ù', 'u', text)
    text = re.sub(r"[0-9]", "", text)

    #text = re.sub(r"Motivo de llamada:\s*(.*)", r'\1', text)

   
    #text=re.sub(r'^.*?(MOTIVO:)', r'\1', text)

    text=re.sub(r'mypimes', " ", text)


    text=re.sub(r'llama da', "llamada", text)
    text=re.sub(r'valdiar', "validar", text)

    #text=re.sub(r'^.*?desea validar\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?se comunica\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?peticion:\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?cliente desea\s*',"",text, flags=re.IGNORECASE)


    text=re.sub(r'^.*?cliente requiere\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?cuenta de facturacion\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?cliente se comunica\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?solicitud del cliente\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?cuenta:\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?cedula de ciudadania\s*',"",text, flags=re.IGNORECASE)
    #text=re.sub(r'^.*?//cliente\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?aseg//*',"",text, flags=re.IGNORECASE)
   
    text=re.sub(r'^.*?motivo de retiro\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?motivo de llamada\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?motivo de la llamada\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?motivo\s*',"",text, flags=re.IGNORECASE)
    text=re.sub(r'^.*?llamada:\s*',"",text, flags=re.IGNORECASE)

    text=re.sub(r'^.*?cliente quiere\s*',"",text, flags=re.IGNORECASE)
    
    text=re.sub(r'^.*?el indica*',"",text, flags=re.IGNORECASE)
    
    

    text=re.sub(r'^.*?ap\s*',"",text, flags=re.IGNORECASE)





    
    #text=re.sub(r'^.*?(ASUNTO:)', r'\1', text)
    #text=re.sub(r'^.*?(descripcion:)', r'\1', text)
    #text=re.sub(r'^.*?(\\ cliente)', r'\1', text)


    #text=re.sub(r'^.*?(FACTURADOS,)', r'\1', text)
    #text=re.sub(r'^.*?(APP MIETB//)', r'\1', text)

    

    #text=re.sub(r'\baseg lt\b', '', text)
    
  


 
    text = re.sub(r"linea", "", text)
    
    text = re.sub(r"LEY 1755", "", text)
    text = re.sub(r"Ley 1755", "", text)
    #text = re.sub(r"ID", "", text)
    
    text = re.sub(r"numeor", "", text)
    text = re.sub(r"MDM-FIBRA-", "", text)

    #text=re.sub(r'^.*?de la llamada\s*', "", text)


    text = re.sub(r'linea', '', text)
    text = re.sub(r'back', '', text)
    text = re.sub(r'Cuenta', '', text)
    text = re.sub(r'CELULAR', '', text)
    text = re.sub(r'Nombre', '', text)
    text = re.sub(r'CLIENTE', '', text)
    text = re.sub(r'RF-', '', text)
    text = re.sub(r'Notas de Escalamiento', '', text)
    text = re.sub(r'Milena', '', text)
    text = re.sub(r'x000D', '', text)
    text = re.sub(r'Asg_Comdata_Caso_', '', text)
    text = re.sub(r'CLT ', '', text)
    text = re.sub(r'GESTION PROACTIVA', '', text)
    text = re.sub(r'lte', '', text)
    text = re.sub(r'whatsapp', '', text)
   
    text = re.sub(r'DOCUMENTO', '', text)
    text = re.sub(r'Número contacto', '', text)
    text = re.sub(r'NA', '', text)
    text = re.sub(r'Motivo de la llamada:', '', text)
    text = re.sub(r'N/A', '', text)
    text = re.sub(r'facebook', '', text)
    text = re.sub(r'TWNogard_ilustra', '', text)
    text = re.sub(r'RRSS', '', text)
    text = re.sub(r'TITULAR', '', text)
    text = re.sub(r'MOTIVO RETIRO', '', text)
    text = re.sub(r'MIXTO', '', text)
    text = re.sub(r'razón', 'razon', text)

    text = re.sub(r"reinicia", "reiniciar", text)
    text = re.sub(r"reset", "reiniciar", text)
    text = re.sub(r"arear", "area", text)
    text = re.sub(r"trasfierir", "transferir", text)
    
 
    text = re.sub(r"tegnicas", "tecnicas", text)
    
    text = re.sub(r"visita tecnica", " visita", text)
    text = re.sub(r"reiniciarr", "reiniciar", text)
    text = re.sub(r"celular", "", text)
    text = re.sub(r"servicios", "servicio", text)
    text = re.sub(r"wifi", "wiffi", text)
    text = re.sub(r"kr", " ", text)
    text = re.sub(r"validan", "valida", text)
   
    #text = re.sub(r"go", "directvgo", text)
    text = re.sub(r"agendamiento", "agenda", text)
    text = re.sub(r"moden", "modem", text)
    text = re.sub(r"/", " ", text)

    text = re.sub(r"información", "informacion", text)
    text = re.sub(r"explicacion", "informacion", text)
    text = re.sub(r"explicación", "informacion", text)
    text = re.sub(r"explicar", "informacion", text)
    
    

     

    
    

    
    
    text = re.sub(r"Gracias por preferirnos y darnos la confianza de ser su proveedor de comunicaciones, estamos seguros de que ha escogido la mejor opción. En ETB trabajamos día a día para que nuestros productos y servicios sean de la mejor calidad, para usted y su Familia", "", text)
    
    #text = re.sub(r"reinicia", "reiniciar", text)
  
    

    #text = re.sub(r"$", "", text)

    
    text = re.sub(r"primer nivel", "primernivel", text)
    text = re.sub(r"soporte primer nivel", "primernivel", text)
    text = re.sub(r" ________________________________________", "", text)
    text = re.sub(r"__________________________", "", text)
    #text=re.sub("(\s\d+)"," ",text)
    text = re.sub(r"cc", " ", text)
    text = re.sub(r"semáforo", "semaforo", text)
    text = re.sub(r"ONT NO ENCIENDE / FALLA", "semaforo", text)
    text = re.sub(r"ak", " ", text)
    text = re.sub(r"sglt", "", text)
    text = re.sub(r"bajo cun", "", text)
    text = re.sub(r"cuenta", "", text)
    text = re.sub(r"radica queja el", "", text)
    text = re.sub(r"\xa0justificacion", "", text)
    text = re.sub(r"poruqe", "porque", text)
    text = re.sub(r"cleitne", "cliente", text)
    text = re.sub(r"indciando", "indicando", text)
    text = re.sub(r"swrvicio", "servicio", text)
    text = re.sub(r"instalaon", "instalaron", text)
    text = re.sub(r"medianete", "mediante", text)
    #text = re.sub(r"inf", "informacion", text)


  #####################################################

    text = re.sub(r"nov", "", text)
    #text = re.sub(r"lica", "aplica", text)
               

    #text=re.sub(r'artir', "", text)


    
    text=re.sub(r'redes sociales', "", text)
    text=re.sub(r'informacipon', "informacion", text)
    text=re.sub(r'sur', "", text)
    
    text=re.sub(r'fac turacion', "facturacion", text)
    text=re.sub(r'de conexion', "", text)
    text=re.sub(r'documento', "", text)
    text=re.sub(r'numero', "", text)
    text=re.sub(r'sep', "", text)
    text=re.sub(r'oct', "", text)
    text=re.sub(r'mdmfibra', "", text)
    text=re.sub(r'revchain', "", text)
    text=re.sub(r'mdm', "", text)
    text=re.sub(r'mes', "", text)
    text=re.sub(r'fecbrero', "", text)
    text=re.sub(r'solictando', "solicitando", text)
    
    text=re.sub(r'numero de documento', "", text)
    text=re.sub(r'numero de conexion', "", text)
    text=re.sub(r'numero de cuenta', "", text)

    #######################dirtect tv y hbo max
    text=re.sub(r'directv go', "directv", text)
    text=re.sub(r'directivi go', "directv", text)
    text=re.sub(r'dtv go', "directv", text)
    #text=re.sub(r'paramount', "par", text)
    

    text=re.sub(r'win sports', "winsport", text)
    text=re.sub(r'win sports+', "winsport", text)
    text=re.sub(r'wing sport', "winsport", text)
    text=re.sub(r'win', "winsport", text)
  
    text=re.sub(r'sva', "svas", text)

    text=re.sub(r'barragan', "", text)

    text=re.sub(r'comun ica', "comunica", text)



    text=re.sub(r'hbo max', "hbo", text)

    
    text=re.sub(r'omunica', "comunica", text)


    ###esta puede estar sujeta a cambio
    text=re.sub(r'explicacion', "informacion", text)

    text=re.sub(r'lalmda', "llamada", text)
    ########
    text = re.sub(r'\binfo\b', 'informacion', text)
    text = re.sub(r'\binf\b', 'informacion', text)
    text = re.sub(r'\bfac\b', 'factura', text)
    #text = re.sub(r'informaci', 'informacion', text)
    text = re.sub(r"ntiene", "tiene", text)
    text = re.sub(r"p. m.", "", text)

    text = re.sub(r"in formacion", "informacion", text)
    text = re.sub(r"informes", "informacion", text)
    
    text = re.sub(r"por que", "porque", text)
    text = re.sub(r"ley", "", text)

    text = re.sub(r"por que", "porque", text)
    text = re.sub(r"actulaizacionde", "actualizacion", text)

    text = re.sub(r"d ela", "de la", text)

    #numero documento
    #numero de conexion
    #numero de cuenta
    
    text=re.sub(r'deyavargasyahoocom', "", text)
    text=re.sub(r'%', "", text)
   



    text = re.sub(r'[-()\"#/@;:%<>+{}`+=~|.!?,]\|_', "", text)
    text=text.replace("[\'","").replace("\n"," ").replace("']"," ").replace('["',"").replace('"]',"").replace("/","").replace("\', \' ;'","")
    text=text.replace("-","").replace(","," ").replace('.'," ").replace('+',"").replace('('," ").replace(')'," ")
    text=text.replace('",'," ").replace("\',","").replace( ":\',","").replace("!","").replace(":","").replace(',"',"")
    text=text.replace("[","").replace("]","").replace("\'s","'s")
    text = text.replace("\'s", "")
    
    text=re.sub(r'rf', "", text)
    text=re.sub(r'\xa0solucion', "", text)
    text=re.sub(r'estoyn', "estoy", text)
    text=re.sub(r'_', "", text)
    
    


    return text

entro2


In [85]:
#declare answers and questions
descripcion=pandas_c_F.DESCRIPCION_PQR	



# Cleaning the questions

clean_descripcion = []
for des in descripcion:
    clean_descripcion.append(clean_text(des))


#restar dos para optener la data de excel y poder comparar

clean_descripcion  [1002]

'  quiere saber porque llego la factura mas cara'

In [86]:
clean_descripcion[883]

'cancela el directv y manifiesta que no va agenrar ningun pago por algo que es negligencia de la empresa'

In [69]:
from tqdm import tqdm
from spelling import correct_sentence

hola


In [87]:
corrected_sentences = []
for text in tqdm(clean_descripcion, desc="Processing"):
    corrected_sentence = correct_sentence(text)
    corrected_sentences.append(corrected_sentence)

Processing: 100%|██████████| 16732/16732 [25:01<00:00, 11.14it/s] 


In [88]:
corrected_sentences[22]

'cliente con dudas de cobro de la factura'

In [89]:
corrected_sentences1=corrected_sentences

In [90]:
from lemastem import data_preprocessing


clean_descripcion=data_preprocessing(corrected_sentences1)


clean_descripcion[883]

['cancela', 'directv', 'ir', 'agendar', 'ningun', 'pago', 'negligencia']

In [91]:
clean_descripcion[863]

['concepto', 'facturar']

In [93]:
import pickle



# Specify the file path where you want to save the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary write mode
with open(file_path, "wb") as file:
    # Use the pickle.dump() function to save the list to the file
    pickle.dump(clean_descripcion, file)

In [ ]:
import pickle

# Specify the file path from where you want to load the list
file_path = "/content/PRIORIZAR/data/entrenamiento_binario.pkl"

# Open the file in binary read mode
with open(file_path, "rb") as file:
    # Use the pickle.load() function to load the list from the file
    loaded_list = pickle.load(file)

print(loaded_list)

In [96]:

clean_descripcion=loaded_list

In [97]:
len(clean_descripcion)

16732

creacion tabla

In [98]:
import numpy as np
import pandas as pd
from unidecode import unidecode



merged_data=[]
for i in range(len(clean_descripcion)):
    merged_data.append(" ".join(clean_descripcion[i]))

merged_df=pd.DataFrame(merged_data,columns =['descripcion']).reset_index()

#############################################


merged_df = pd.merge(merged_df, pandas_c_F, on='index', how='inner').loc[:, ['FECHA_CREACION','NUMERO_PQR','descripcion', 'clasificacion']]


shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)




#shuffled_df.rename(columns={"FECHA_CREACION": "event_timestamp"}, inplace=True)

#string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']



# Función para eliminar las tildes de un texto
def remove_accents(text):
    return unidecode(text)

# Aplicar la función a la columna 'Texto'
shuffled_df['descripcion'] = shuffled_df['descripcion'].apply(remove_accents)



# Randomly select string values to fill the DataFrame
#random_strings = np.random.choice(string_values, size=len(shuffled_df))

#shuffled_df["NUMERO_PQR"]=random_strings


shuffled_df

,FECHA_CREACION,NUMERO_PQR,descripcion,clasificacion
0,2023-01-11 13:04:52,MDM-PQR-36237861,unicar aclarar concepto facturar factura llego...,0
1,2023-02-07 23:28:25,MDM-PQR-36939031,categoriaplato telefono fijo motivo d llamado ...,0
2,2022-12-06 07:30:57,MDM-PQR-35530002,factura,1
3,2022-12-29 09:43:19,MDM-PQR-35955405,llego factura solicitar saber,1
4,2023-01-23 09:28:45,MDM-PQR-36493499,unicar consultar si poder pagar factura medio ...,0
...,...,...,...,...
16727,2023-02-09 13:47:06,MDM-PQR-36979852,unicar confirmar aumento factura,1
16728,2023-02-01 09:56:43,MDM-PQR-36752371,unicar confirmar aumento factura,1
16729,2023-01-18 13:31:05,MDM-PQR-36394327,llamda plan telefonia,1
16730,2023-01-30 11:16:29,MDM-PQR-36689010,unicar llego factura llego pensar promocion,0


In [99]:
shuffled_df.to_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv', index=False)

In [9]:
shuffled_df = pd.read_csv('/content/PRIORIZAR/entranamiento_/entrana_bina.csv')

In [100]:
from tfidf import TFIDF

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from joblib import dump



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion']




# train test split (66% train - 33% test)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)




X_train,X_test = TFIDF(X_train,X_test)

pca = PCA()

pca.fit(X_train)

# Get the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_

# Calculate the cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

# Find the number of components that explain a desired amount of variance
desired_variance = 0.9  # Set the desired variance threshold
n_components = np.argmax(cumulative_variance_ratio >= desired_variance) + 1

# Create a new instance of PCA with the optimal number of components
pca = PCA(n_components=n_components)


X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)



print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)


dump(value=pca, filename="/content/PRIORIZAR/modelo/pca_binario.joblib")

tf-idf with 2760 features
train with old features:  (13385, 2760)
train with new features: (13385, 534)
test with old features:  (3347, 2760)
test with new features: (3347, 534)
Training Data : (13385, 2760)
Testing Data :  (3347, 2760)


['/content/PRIORIZAR/modelo/pca_binario.joblib']

In [102]:
n_components

534

In [103]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 7.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.3 MB/s eta 0:00:00


In [104]:
import optuna
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score


In [105]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimators": trial.suggest_int("n_estimators", 20, 200),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 5.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 5.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 30, 90),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'learning_rate': trial.suggest_float("learning_rate", 0.04, 0.1),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }




    gbm = lgb.LGBMClassifier(**param)
    gbm.fit(X_train_new, y_train)
    preds = gbm.predict(X_test_new)
    accuracy = accuracy_score(y_test, preds)
    return accuracy




In [106]:
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

[I 2023-06-23 03:41:38,168] A new study created in memory with name: lightgbm


[LightGBM] [Warning] bagging_fraction is set=0.4554031568612787, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4554031568612787
[LightGBM] [Warning] lambda_l2 is set=0.010007110473405195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010007110473405195
[LightGBM] [Warning] lambda_l1 is set=0.879294185249291, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.879294185249291
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.48805353449026784, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48805353449026784


[I 2023-06-23 03:41:44,949] Trial 0 finished with value: 0.5945622945921721 and parameters: {'n_estimators': 95, 'lambda_l1': 0.879294185249291, 'lambda_l2': 0.010007110473405195, 'num_leaves': 48, 'feature_fraction': 0.48805353449026784, 'bagging_fraction': 0.4554031568612787, 'bagging_freq': 2, 'learning_rate': 0.060733643622582864, 'min_child_samples': 43}. Best is trial 0 with value: 0.5945622945921721.


[LightGBM] [Warning] bagging_fraction is set=0.4164325559187557, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4164325559187557
[LightGBM] [Warning] lambda_l2 is set=0.7069470359901728, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7069470359901728
[LightGBM] [Warning] lambda_l1 is set=0.1353297938831623, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.1353297938831623
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9268704618345672, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9268704618345672


[I 2023-06-23 03:41:53,973] Trial 1 finished with value: 0.5912757693456827 and parameters: {'n_estimators': 117, 'lambda_l1': 0.1353297938831623, 'lambda_l2': 0.7069470359901728, 'num_leaves': 42, 'feature_fraction': 0.9268704618345672, 'bagging_fraction': 0.4164325559187557, 'bagging_freq': 5, 'learning_rate': 0.06503828814202763, 'min_child_samples': 58}. Best is trial 0 with value: 0.5945622945921721.


...

[LightGBM] [Warning] bagging_fraction is set=0.6086595158473039, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6086595158473039
[LightGBM] [Warning] lambda_l2 is set=0.11839077252754517, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.11839077252754517
[LightGBM] [Warning] lambda_l1 is set=1.510514400536787, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.510514400536787
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] feature_fraction is set=0.9565051482376203, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9565051482376203


[I 2023-06-23 03:43:06,771] Trial 8 finished with value: 0.5909769943232746 and parameters: {'n_estimators': 45, 'lambda_l1': 1.510514400536787, 'lambda_l2': 0.11839077252754517, 'num_leaves': 40, 'feature_fraction': 0.9565051482376203, 'bagging_fraction': 0.6086595158473039, 'bagging_freq': 6, 'learning_rate': 0.0835598791210271, 'min_child_samples': 89}. Best is trial 6 with value: 0.5996414699731103.


[LightGBM] [Warning] bagging_fraction is set=0.656854713922777, subsample=1.0 will be ignored. Current value: bagging_fraction=0.656854713922777
[LightGBM] [Warning] lambda_l2 is set=0.08743114371278075, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.08743114371278075
[LightGBM] [Warning] lambda_l1 is set=1.0635823077784037, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.0635823077784037
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.9375317309176401, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9375317309176401


[I 2023-06-23 03:43:18,283] Trial 9 finished with value: 0.5936659695249478 and parameters: {'n_estimators': 132, 'lambda_l1': 1.0635823077784037, 'lambda_l2': 0.08743114371278075, 'num_leaves': 46, 'feature_fraction': 0.9375317309176401, 'bagging_fraction': 0.656854713922777, 'bagging_freq': 7, 'learning_rate': 0.07980648986910689, 'min_child_samples': 64}. Best is trial 6 with value: 0.5996414699731103.


[LightGBM] [Warning] bagging_fraction is set=0.7703756492982702, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7703756492982702
[LightGBM] [Warning] lambda_l2 is set=4.077848194578335, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.077848194578335
[LightGBM] [Warning] lambda_l1 is set=0.12917970593385644, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.12917970593385644
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] feature_fraction is set=0.77592993314995, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.77592993314995


In [51]:
print('Best parameters:', study.best_params)

Best parameters: {'n_estimators': 307, 'lambda_l1': 0.013932335006314283, 'lambda_l2': 0.35442650826863986, 'num_leaves': 136, 'feature_fraction': 0.7535833221419705, 'bagging_fraction': 0.8198550160125587, 'bagging_freq': 1, 'learning_rate': 0.0648433592691741, 'min_child_samples': 71}


In [52]:
mode = lgb.LGBMClassifier(**study.best_params)
mode.fit(X_train_new, y_train)

[LightGBM] [Warning] lambda_l1 is set=0.013932335006314283, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.013932335006314283
[LightGBM] [Warning] feature_fraction is set=0.7535833221419705, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7535833221419705
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.35442650826863986, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.35442650826863986
[LightGBM] [Warning] bagging_fraction is set=0.8198550160125587, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8198550160125587


LGBMClassifier(bagging_fraction=0.8198550160125587, bagging_freq=1,
               feature_fraction=0.7535833221419705,
               lambda_l1=0.013932335006314283, lambda_l2=0.35442650826863986,
               learning_rate=0.0648433592691741, min_child_samples=71,
               n_estimators=307, num_leaves=136)

In [55]:
y_pred = mode.predict(X_test_new)

In [56]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.60      0.60      0.60      1679
           1       0.60      0.60      0.60      1666

    accuracy                           0.60      3345
   macro avg       0.60      0.60      0.60      3345
weighted avg       0.60      0.60      0.60      3345



In [60]:
from joblib import dump
dump(value=mode, filename="content/modelo_binario.joblib")

['content/modelo_binario.joblib']

para data a predecir

modelo creacion

In [72]:

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.decomposition import PCA



X = shuffled_df['descripcion']

y = shuffled_df['clasificacion'].astype(int)




# train test split (66% train - 33% test)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)






def TFIDF(X_train, X_test, MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with", str(np.array(X_train).shape[1]), "features")
    return (X_train, X_test)


X_train,X_test = TFIDF(X_train,X_test)



pca = PCA(n_components=5000)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)





print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)

tf-idf with 6404 features
train with old features:  (13385, 6404)
train with new features: (13385, 5000)
test with old features:  (3347, 6404)
test with new features: (3347, 5000)
Training Data : (13385, 6404)
Testing Data :  (3347, 6404)


In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

lr = LogisticRegression()



lr.fit(X_train_new, y_train)



# generate predictions

predictions = lr.predict(X_test_new)

predictions

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['1', '0', '1', ..., '0', '0', '0'], dtype=object)

In [74]:
import lightgbm as lgb
from joblib import dump

train_data = lgb.Dataset(X_train_new, label=y_train)

# Set hyperparameters for LightGBM
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the LightGBM model
model = lgb.train(params, train_data, num_boost_round=100)

# Make predictions on the test set
y_pred = model.predict(X_test_new)

y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)


from sklearn import metrics

df = pd.DataFrame(metrics.confusion_matrix(y_test,y_pred_binary), index=['OK','WRONG'], columns=['OK','WRONG'])

df

Accuracy: 0.9214221691066626


In [101]:
dump(value=model, filename="model.joblib")

['model.joblib']

predicciendo data nueva

In [95]:
store = FeatureStore(repo_path="/my_project/feature_repo/")

t_predict = store.get_saved_dataset(name="prediccion_2").to_df()

t_predict

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/feast/feature_store.py:1230: RuntimeWarning: Retrieving datasets is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


,descripcion,NUMERO_PQR,event_timestamp
0,concepto facturabl valor factura,fghij,2023-03-01 00:39:28+00:00
1,concepto facturabl valor factura,uvwxy,2023-03-01 00:39:28+00:00
2,concepto facturabl valor factura,fghij,2023-03-01 05:12:45+00:00
3,concepto facturabl valor factura,abcde,2023-03-01 05:12:45+00:00
4,cleitir comuncia indicar completo orden caibo ...,jklmn,2023-03-01 06:27:05+00:00
...,...,...,...
25622,queire validar venefiico servicio costo,klmno,2023-05-21 18:08:46+00:00
25623,explicacion factura solucion pqr,zabcd,2023-05-21 18:13:34+00:00
25624,solicitar informacion factura,abcde,2023-05-21 18:33:20+00:00
25625,generar duplicado factura,opqrs,2023-05-21 18:39:27+00:00


In [126]:
X_train = merged_df['descripcion']





def TFIDF(X_train, X_test, MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with", str(np.array(X_train).shape[1]), "features")
    return (X_train, X_test)


X_train,X_test = TFIDF(X_train,_)





pca = PCA(n_components=5000)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

print("train with old features: ",np.array(X_train).shape)
print("train with new features:" ,np.array(X_train_new).shape)

print("test with old features: ",np.array(X_test).shape)
print("test with new features:" ,np.array(X_test_new).shape)



print('Training Data :', X_train.shape)

print('Testing Data : ', X_test.shape)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tf-idf with 9935 features
train with old features:  (25694, 9935)
train with new features: (25694, 5000)
test with old features:  (7, 9935)
test with new features: (7, 5000)
Training Data : (25694, 9935)
Testing Data :  (7, 9935)


In [130]:
y_pred = model.predict(X_train_new)

y_pred_binary = [1 if val >= 0.5 else 0 for val in y_pred]


In [133]:
y_pred_binary

[1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,


In [134]:

pandas_c_F['prediccion_algoritmo'] = y_pred_binary

pandas_c_F

,level_0,index,FECHA_CREACION,NUMERO_PQR,DESCRIPCION_PQR,prediccion_algoritmo
0,0,0,2023-03-01 00:39:28,MDM-PQR-37404417,SE COMUNICA POR CONCEPTOS FACTURABLES Y VALOR ...,1
1,1,1,2023-03-01 05:12:45,MDM-PQR-37404567,SE COMUNICA POR CONCEPTOS FACTURABLES Y VALOR ...,1
2,2,2,2023-03-01 06:27:05,MDM-PQR-37404722,Línea: 6018797137 Nombre de quien se comunic...,1
3,3,3,2023-03-01 06:33:27,MDM-PQR-37404741,NOMBRE: NELSY YADIRA MORENO CASTIBLANCO ID: ...,1
4,4,4,2023-03-01 06:56:09,MDM-PQR-37404860,CC:1000727176 NOMBRE:ANGELY K TALINA OSORIO VA...,1
...,...,...,...,...,...,...
25689,25694,25694,2023-05-21 19:41:16,MDM-PQR-38950393,queire validar los venefiicos del servicio y ...,1
25690,25695,25695,2023-05-21 19:47:48,MDM-PQR-38950430,NOMBRE: FRANCISCO PUENTES PUENTES CC: 794365...,1
25691,25696,25696,2023-05-21 20:00:34,MDM-PQR-38950481,Solicita información de factura,1
25692,25697,25697,2023-05-21 21:21:29,MDM-PQR-38950811,CLIENTE SE COMUNICA PARA GENERAR DUPLICADO DE ...,0


In [135]:
pandas_c_F.to_csv('c_facturados.csv', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
from feast import FeatureStore
from datetime import datetime, timedelta

# Getting our FeatureStore
store = FeatureStore(repo_path="/my_project/feature_repo/")

# Code for loading features to online store between two dates
"""store.materialize(
    end_date=datetime.now(),
    start_date=datetime.now() - timedelta(days=900))"""

# Loading the latest features after a previous materialize call or from the beginning of time
#store.materialize_incremental(end_date=datetime.now())


store.materialize(
    end_date=datetime.now(),
    start_date=partida)

Materializing 1 feature views from 2023-03-25 19:24:48+00:00 to 2023-05-24 19:24:50+00:00 into the sqlite online store.

prediccion:


100%|█████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2106.42it/s]


In [10]:
from datetime import datetime, timedelta



partida=datetime.now() - timedelta(days=60)

# Print the future date
print(f"Current Date: {partida}")


Current Date: 2023-03-25 19:24:48.466720


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
feast_features = [
        "prediccion:descripcion"
    ]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
features = store.get_online_features(
    features=feast_features,    
    entity_rows=[{"NUMERO_PQR": "abcde"}]
).to_dict()

string_values = ['abcde', 'fghij', 'klmno', 'pqrst', 'uvwxy', 'zabcd', 'efghi', 'jklmn', 'opqrs', 'tuvwx']

In [14]:
features

{'NUMERO_PQR': ['abcde'], 'descripcion': ['solicitar informacion factura']}

In [13]:
features_df = pd.DataFrame.from_dict(data=features)

features_df

,NUMERO_PQR,descripcion
0,,None
